## Initializing of XLS FILE INTO DATAFRAME FORMAT

In [ ]:
import pandas as pd

df = pd.read_excel('PPD_Transition_Energetique_Scrapping.xlsx')

df

## CLEANING - EXTRA SPACES, PUNCTUATIONS, TABULATION

In [ ]:
#!pip install clean-text --user 
import re
import string
import nltk
from nltk.tokenize import TweetTokenizer
import emoji

#nltk.download('punkt')

def delete_extra_spaces(txt):
    txt = re.sub(' +', ' ', txt)
    return txt

def delete_punct(txt):
    #txt = re.sub("[^-9A-Za-z ]", "" , txt)
    txt = "".join([i for i in txt if i not in string.punctuation])
    #txt = txt.translate(str.maketrans('', '', string.punctuation))
    #txt = re.sub(r'[^\w\s]', '', txt)
    #txt = ''.join(filter(lambda x: x.isalpha() or x.isdigit() or x.isspace(), txt))
    return txt

def set_lower_case(txt):
    txt = "".join([i.lower() for i in txt if i not in string.punctuation])
    return txt

def tokenizing_nltk(txt):
    if not txt:
        print('Test, cellules vides')
        text = ''
    txt = nltk.tokenize.word_tokenize(txt)
    tweet = TweetTokenizer()
    #tweet.tokenize(txt)    
    return txt


df["Content"] = df["Content"].apply(delete_extra_spaces)
df["Content"] = df["Content"].apply(delete_punct)
df["Content"] = df["Content"].apply(set_lower_case)
#df["Content"] = df["Content"].apply(lambda s: ' '.join(filter(lambda c: c.isalpha(), s)))
df['Content'] = df['Content'].str.replace('[^\w\s#@/:%.,_-]', ' ', flags=re.UNICODE)
df["Content"].dropna(inplace=True)

df["Content"] = df["Content"].apply(tokenizing_nltk)

#df["Content"] = df["Content"].apply(lambda s: emoji.replace_emoji(s, ' '))

df["Content"]

In [ ]:
def get_all_words():
    tmp=[]
    for tweet in df["Content"]:
        for txt in tweet:
            tmp.append(txt)
         
    return tmp




def tokenizing_nltkplus(txt):
    if not txt:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    #txt = nltk.tokenize.word_tokenize(txt)
    tweet = TweetTokenizer()
    tweet.tokenize(txt)    
    return txt

stringwords = get_all_words()

def get_refactored_words():
    tab = []
    for ele in stringwords:
        tokenizing_nltkplus(ele)
        
        tab.append(ele)
    return tab


clap = get_refactored_words()

clap

### WORK ON STOPWORDS

In [ ]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('french')

In [ ]:
dfst = pd.read_excel('stopwords-fr.xlsx', encoding='utf-8')

def get_all_stopwords():
    tb=[]
    for stp in dfst["stopwords-fr"]:
        tb.append(stp)
        
    return tb

stopwords_fr = get_all_stopwords()

In [ ]:
stopwords_all = nltk.corpus.stopwords.words('english') + stopwords_fr
#text_retestponc = " ".join([i for i in get_refactored_words() if i not in string.punctuation])
text_retestponc = " ".join([i for i in get_refactored_words()])
#text_without_digits = ''.join((x for x in text_retestponc if not x.isdigit()))
#print(text_retestponc)


In [ ]:
mots_tokens = nltk.tokenize.word_tokenize(text_retestponc)
print(mots_tokens)


In [ ]:
mots_without_stopwords = [i for i in mots_tokens if i not in stopwords_all]

print(mots_without_stopwords)

In [ ]:
ss = nltk.SnowballStemmer(language='french')
mt =  [ss.stem(mot) for mot in mots_without_stopwords]
print(mt)

In [ ]:
nltk.download('wordnet')
motlemma = nltk.WordNetLemmatizer()
motlemma = [motlemma.lemmatize(mot) for mot in mots_without_stopwords]
print(motlemma)

In [ ]:
from nltk.probability import FreqDist
pd.set_option("max_rows", 500)

freq_dis = FreqDist(motlemma)
freq = pd.DataFrame(list(freq_dis.items()), columns=['mots', 'fréquence d\'apparition'])

freq['fréquence d\'apparition'] = pd.to_numeric(freq['fréquence d\'apparition'], errors='coerce')
freq_new = freq.sort_values('fréquence d\'apparition', ascending=False)
freq_new.iloc[:350]

freq_new.to_csv('20230425 Dictionnaire de mots - Transition Energetique.csv', encoding='utf-8', index=False)
